In [1]:
import keras
from keras.applications import VGG19
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
IMAGE_SIZE = [224, 224]

train_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training'
test_path = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing'

In [4]:
vgg19_model = VGG19(input_shape=IMAGE_SIZE + [3],weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 3s 0us/step


In [5]:
for layer in vgg19_model.layers:
    layer.trainable = False

In [6]:
folders = glob('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training/*')
x = Flatten()(vgg19_model.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg19_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 4600 images belonging to 2 classes.
Found 1872 images belonging to 2 classes.


In [11]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

460/460 [==============================] - 2961s 6s/step - loss: 0.3623 - accuracy: 0.8428 - val_loss: 0.2018 - val_accuracy: 0.9231


In [12]:
import tensorflow as tf
from keras.models import load_model

model.save('Brain Tumour1.h5')